In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
titanic=pd.read_csv('/kaggle/input/titanic/train.csv',index_col='PassengerId')
titanic_test=pd.read_csv('/kaggle/input/titanic/test.csv',index_col='PassengerId')

titanic.head(10)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S


In [3]:
titanic.isnull().sum()

Survived      0
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [4]:
titanic_test.isnull().sum()

Pclass        0
Name          0
Sex           0
Age          86
SibSp         0
Parch         0
Ticket        0
Fare          1
Cabin       327
Embarked      0
dtype: int64

In [5]:
titanic=titanic.drop('Cabin',axis=1)
titanic_test=titanic_test.drop('Cabin',axis=1)
titanic.Embarked.value_counts()
# Filling the two missing values with the most frequent value for Embarked column
titanic[['Embarked']]=titanic[['Embarked']].fillna('S')

In [6]:
titanic.groupby('Pclass')['Fare'].mean()

Pclass
1    84.154687
2    20.662183
3    13.675550
Name: Fare, dtype: float64

In [7]:
titanic_test[titanic_test['Fare'].isnull()==True]
titanic_test.loc[titanic_test['Fare'].isnull()==True,'Fare']=13.6756

In [8]:
# Creating a list for saving the title of people form the name column
var=titanic['Name'].str.split()
list_of_title=[]
for x in range(1,(len(titanic)+1)):
    list_of_title.append(var[x][1])

In [9]:
titanic['Title']=list_of_title
titanic.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Title
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,Mr.
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,Mrs.
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,Miss.
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,Mrs.
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,Mr.


In [10]:
titanic['Title'].unique()

array(['Mr.', 'Mrs.', 'Miss.', 'Master.', 'Planke,', 'Don.', 'Rev.',
       'Billiard,', 'der', 'Walle,', 'Dr.', 'Pelsmaeker,', 'Mulder,', 'y',
       'Steen,', 'Carlo,', 'Mme.', 'Impe,', 'Ms.', 'Major.', 'Gordon,',
       'Messemaeker,', 'Mlle.', 'Col.', 'Capt.', 'Velde,', 'the',
       'Shawah,', 'Jonkheer.', 'Melkebeke,', 'Cruyssen,'], dtype=object)

In [11]:
corrections=['Planke,','Billiard,', 'der', 'Walle,',  'Pelsmaeker,', 'Mulder,', 'y',
       'Steen,', 'Carlo,',  'Gordon,','Messemaeker,',  'Velde,', 'the',
       'Shawah,', 'Jonkheer.', 'Melkebeke,', 'Cruyssen,', 'Khalil,',
       'Palmquist,', 'Brito,']

for x in corrections:
    print(titanic.loc[titanic['Title']==x,'Name'])

PassengerId
19     Vander Planke, Mrs. Julius (Emelia Maria Vande...
39                    Vander Planke, Miss. Augusta Maria
334                      Vander Planke, Mr. Leo Edmondus
Name: Name, dtype: object
PassengerId
154    van Billiard, Mr. Austin Blyler
Name: Name, dtype: object
PassengerId
171    Van der hoef, Mr. Wyckoff
Name: Name, dtype: object
PassengerId
201    Vande Walle, Mr. Nestor Cyriel
Name: Name, dtype: object
PassengerId
283    de Pelsmaeker, Mr. Alfons
Name: Name, dtype: object
PassengerId
287    de Mulder, Mr. Theodore
Name: Name, dtype: object
PassengerId
308    Penasco y Castellana, Mrs. Victor de Satode (M...
506           Penasco y Castellana, Mr. Victor de Satode
548                           Padro y Manent, Mr. Julian
867                         Duran y More, Miss. Asuncion
Name: Name, dtype: object
PassengerId
356    Vanden Steen, Mr. Leo Peter
Name: Name, dtype: object
PassengerId
362    del Carlo, Mr. Sebastiano
Name: Name, dtype: object
PassengerId
557  

In [12]:
#Some titles needed some modifications based on their names

two=[19,39,334,154,201,283,287,356,362,420,596,800,560,753,760,799,869,874]
three=[171,308,506,548,867]
for x in two:
    titanic.loc[x,'Title']=var[x][2]
    
for y in three:
    titanic.loc[y,'Title']=var[y][3]
    
    
titanic.loc[557,'Title']='Mrs'
titanic.loc[600,'Title']='Mr'
titanic.loc[760,'Title']='Mrs'

#Some of the titles like Jonkheer,Col,Capt,etc are being left as it is.

In [13]:
# Creating a list for saving the title of people form the name column
var=titanic_test['Name'].str.split()
list_of_title=[]
for x in range(titanic_test.index[0],(titanic_test.index[-1]+1)):
    list_of_title.append(var[x][1])
titanic_test['Title']=list_of_title
titanic_test.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Title
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,Q,Mr.
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,S,Mrs.
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,Q,Mr.
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,S,Mr.
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,S,Mrs.


In [14]:
titanic_test['Title'].unique()

array(['Mr.', 'Mrs.', 'Miss.', 'Carlo,', 'Khalil,', 'Master.', 'y', 'Ms.',
       'Palmquist,', 'Col.', 'Planke,', 'Rev.', 'Billiard,',
       'Messemaeker,', 'Dr.', 'Brito,'], dtype=object)

In [15]:
corrections=['Planke,','Billiard,', 'der', 'Walle,',  'Pelsmaeker,','y',
        'Carlo,',  'Gordon,','Messemaeker,',  'Velde,', 'the',
       'Shawah,', 'Jonkheer.', 'Melkebeke,', 'Cruyssen,', 'Khalil,',
       'Palmquist,', 'Brito,']

for x in corrections:
    print(titanic_test.loc[titanic_test['Title']==x,'Name'])

PassengerId
1037    Vander Planke, Mr. Julius
Name: Name, dtype: object
PassengerId
1084      van Billiard, Master. Walter John
1236    van Billiard, Master. James William
Name: Name, dtype: object
Series([], Name: Name, dtype: object)
Series([], Name: Name, dtype: object)
Series([], Name: Name, dtype: object)
PassengerId
965     Ovies y Rodriguez, Mr. Servando
1112     Duran y More, Miss. Florentina
1261      Pallas y Castello, Mr. Emilio
1306       Oliva y Ocana, Dona. Fermina
Name: Name, dtype: object
PassengerId
907    del Carlo, Mrs. Sebastiano (Argenia Genovesi)
Name: Name, dtype: object
Series([], Name: Name, dtype: object)
PassengerId
1152    de Messemaeker, Mr. Guillaume Joseph
Name: Name, dtype: object
Series([], Name: Name, dtype: object)
Series([], Name: Name, dtype: object)
Series([], Name: Name, dtype: object)
Series([], Name: Name, dtype: object)
Series([], Name: Name, dtype: object)
Series([], Name: Name, dtype: object)
PassengerId
911    Assaf Khalil, Mrs. Mariana (Mir

In [16]:
#Some titles needed some modifications based on their names

two_test=[1037,1084,1236,907,1152,911,995,1228]
three_test=[965,1112,1261,1306]
for x in two_test:
    titanic_test.loc[x,'Title']=var[x][2]
    
for y in three_test:
    titanic_test.loc[y,'Title']=var[y][3]
    
    

#Some of the titles like Jonkheer,Col,Capt,etc are being left as it is.

In [17]:
titanic['Title']=titanic['Title'].str.replace('.','').str.replace(',','')
titanic_test['Title']=titanic_test['Title'].str.replace('.','').str.replace(',','')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  


In [18]:
# The 'Ms' title has only two rows and one of them is missing, so assigning the value of the first row to the missing one.
titanic_test[titanic_test['Title']=='Ms']

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Title
PassengerId,,,,,,,,,,
980,3,"O'Donoghue, Ms. Bridget",female,NaN,0,0,364856,7.75,Q,Ms


In [19]:
titanic[titanic['Title']=='Ms']

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Title
PassengerId,,,,,,,,,,,
444,1,2,"Reynaldo, Ms. Encarnacion",female,28.0,0,0,230434,13.0,S,Ms


In [20]:
titanic_test.loc[980,'Age']=28.0

In [21]:
mr_age=titanic[titanic['Title']=='Mr']['Age'].median()
master_age=titanic[titanic['Title']=='Master']['Age'].median()
mrs_age=titanic[titanic['Title']=='Mrs']['Age'].median()
miss_age=titanic[titanic['Title']=='Miss']['Age'].median()
dr_age=titanic[titanic['Title']=='Dr']['Age'].median()

In [22]:
titanic.loc[titanic['Title']=='Mr','Age']=mr_age
titanic_test.loc[titanic_test['Title']=='Mr','Age']=mr_age

titanic.loc[titanic['Title']=='Master','Age']=master_age
titanic_test.loc[titanic_test['Title']=='Master','Age']=master_age

titanic.loc[titanic['Title']=='Mrs','Age']=mrs_age
titanic_test.loc[titanic_test['Title']=='Mrs','Age']=mrs_age

titanic.loc[titanic['Title']=='Miss','Age']=miss_age
titanic_test.loc[titanic_test['Title']=='Miss','Age']=miss_age

titanic.loc[titanic['Title']=='Dr','Age']=dr_age
titanic_test.loc[titanic_test['Title']=='Dr','Age']=dr_age

In [23]:
titanic.isnull().sum()

Survived    0
Pclass      0
Name        0
Sex         0
Age         0
SibSp       0
Parch       0
Ticket      0
Fare        0
Embarked    0
Title       0
dtype: int64

In [24]:
titanic_test.isnull().sum()

Pclass      0
Name        0
Sex         0
Age         0
SibSp       0
Parch       0
Ticket      0
Fare        0
Embarked    0
Title       0
dtype: int64

In [25]:
titanic.tail()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Title
PassengerId,,,,,,,,,,,
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.00,S,Rev
888,1,1,"Graham, Miss. Margaret Edith",female,21.0,0,0,112053,30.00,S,Miss
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,21.0,1,2,W./C. 6607,23.45,S,Miss
890,1,1,"Behr, Mr. Karl Howell",male,30.0,0,0,111369,30.00,C,Mr
891,0,3,"Dooley, Mr. Patrick",male,30.0,0,0,370376,7.75,Q,Mr


In [26]:
titanic['Sex']=titanic['Sex'].map({'male':0,'female':1})

titanic_test['Sex']=titanic_test['Sex'].map({'male':0,'female':1})

In [27]:
titanic['Family_members']=titanic['SibSp']+titanic['Parch']
titanic_test['Family_members']=titanic_test['SibSp']+titanic_test['Parch']

In [28]:
titanic=titanic.drop(['Name','Ticket','Title'],axis=1)
titanic_test=titanic_test.drop(['Name','Ticket','Title'],axis=1)

In [29]:
embarked_dummies=pd.get_dummies(titanic.Embarked,prefix='Embarked')
embarked_dummies_test=pd.get_dummies(titanic_test.Embarked,prefix='Embarked')

titanic=pd.concat([titanic,embarked_dummies],axis=1)
titanic_test=pd.concat([titanic_test,embarked_dummies_test],axis=1)

titanic=titanic.drop('Embarked',axis=1)
titanic_test=titanic_test.drop('Embarked',axis=1)

In [30]:
pclass_dummies=pd.get_dummies(titanic.Pclass,prefix='class')
pclass_dummies_test=pd.get_dummies(titanic_test.Pclass,prefix='class')

titanic=pd.concat([titanic,pclass_dummies],axis=1)
titanic_test=pd.concat([titanic_test,pclass_dummies_test],axis=1)

titanic=titanic.drop('Pclass',axis=1)
titanic_test=titanic_test.drop('Pclass',axis=1)

In [31]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# transform data
df=titanic.loc[:,['Age','Fare','Family_members']]
df_test=titanic_test.loc[:,['Age','Fare','Family_members']]
scaled = pd.DataFrame(scaler.fit_transform(df))
scaled_test = pd.DataFrame(scaler.transform(df_test))

In [32]:
scaled.index=titanic.index
scaled.columns=df.columns
scaled_test.index=df_test.index
scaled_test.columns=df_test.columns

In [33]:
titanic.loc[:,['Age','Fare','Family_members']]=scaled.loc[:,['Age','Fare','Family_members']]
titanic_test.loc[:,['Age','Fare','Family_members']]=scaled_test.loc[:,['Age','Fare','Family_members']]

In [34]:
titanic

,Survived,Sex,Age,SibSp,Parch,Fare,Family_members,Embarked_C,Embarked_Q,Embarked_S,class_1,class_2,class_3
PassengerId,,,,,,,,,,,,,
1,0,0,0.258797,1,0,-0.502445,0.059160,0,0,1,0,0,1
2,1,1,0.922361,1,0,0.786845,0.059160,1,0,0,1,0,0
3,1,1,-0.935618,0,0,-0.488854,-0.560975,0,0,1,0,0,1
4,1,1,0.922361,1,0,0.420730,0.059160,0,0,1,1,0,0
5,0,0,0.258797,0,0,-0.486337,-0.560975,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
887,0,0,-0.139341,0,0,-0.386671,-0.560975,0,0,1,0,1,0
888,1,1,-0.935618,0,0,-0.044381,-0.560975,0,0,1,1,0,0
889,0,1,-0.935618,1,2,-0.176263,1.299429,0,0,1,0,0,1


In [35]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(titanic.loc[:,titanic.columns!='Survived'],titanic['Survived'], 
                test_size = 0.20, random_state = 78)

In [36]:
#Logistic Regression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,classification_report, confusion_matrix
lr=LogisticRegression()
lr.fit(X_train,y_train)
predictions=lr.predict(X_test)
acc=accuracy_score(y_test,predictions)
print(acc)
print(classification_report(y_test,predictions))
print(confusion_matrix(y_test,predictions))

0.8324022346368715
              precision    recall  f1-score   support

           0       0.89      0.84      0.87       116
           1       0.74      0.81      0.77        63

    accuracy                           0.83       179
   macro avg       0.82      0.83      0.82       179
weighted avg       0.84      0.83      0.83       179

[[98 18]
 [12 51]]


In [37]:
from sklearn.svm import SVC
svc=SVC(C= 1000, gamma= 0.001,kernel= 'rbf')
svc.fit(X_train,y_train)
predictions=svc.predict(X_test)
acc=accuracy_score(y_test,predictions)
print(acc)
print(classification_report(y_test,predictions))
print(confusion_matrix(y_test,predictions))

0.8770949720670391
              precision    recall  f1-score   support

           0       0.91      0.91      0.91       116
           1       0.83      0.83      0.83        63

    accuracy                           0.88       179
   macro avg       0.87      0.87      0.87       179
weighted avg       0.88      0.88      0.88       179

[[105  11]
 [ 11  52]]


In [38]:
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier(n_estimators=100)
rfc.fit(X_train,y_train)
predictions=rfc.predict(X_test)
acc=accuracy_score(y_test,predictions)
print(acc)
print(classification_report(y_test,predictions))
print(confusion_matrix(y_test,predictions))

0.8044692737430168
              precision    recall  f1-score   support

           0       0.88      0.81      0.84       116
           1       0.69      0.79      0.74        63

    accuracy                           0.80       179
   macro avg       0.79      0.80      0.79       179
weighted avg       0.81      0.80      0.81       179

[[94 22]
 [13 50]]


In [39]:
from xgboost import XGBClassifier
xgbc=XGBClassifier(n_estimators=100,learning_rate=0.1,gamma=0.1,
                   eval_metric='logloss',use_label_encoder=False)
xgbc.fit(X_train, y_train, 
             early_stopping_rounds=5, 
             eval_set=[(X_test, y_test)], 
             verbose=False)
predictions=xgbc.predict(X_test)
acc=accuracy_score(y_test,predictions)
print(acc)
print(classification_report(y_test,predictions))
print(confusion_matrix(y_test,predictions))

0.8491620111731844
              precision    recall  f1-score   support

           0       0.90      0.86      0.88       116
           1       0.76      0.83      0.79        63

    accuracy                           0.85       179
   macro avg       0.83      0.84      0.84       179
weighted avg       0.85      0.85      0.85       179

[[100  16]
 [ 11  52]]


In [40]:
from sklearn.ensemble import GradientBoostingClassifier
gbc=GradientBoostingClassifier(learning_rate= 0.05, max_depth= 4, n_estimators= 100)
gbc.fit(X_train,y_train)
predictions=gbc.predict(X_test)
acc=accuracy_score(y_test,predictions)
print(acc)
print(classification_report(y_test,predictions))
print(confusion_matrix(y_test,predictions))

0.8659217877094972
              precision    recall  f1-score   support

           0       0.90      0.89      0.90       116
           1       0.80      0.83      0.81        63

    accuracy                           0.87       179
   macro avg       0.85      0.86      0.85       179
weighted avg       0.87      0.87      0.87       179

[[103  13]
 [ 11  52]]


In [41]:
preds_test=svc.predict(titanic_test)
print(preds_test)
output = pd.DataFrame({'PassengerId': titanic_test.index,
                       'Survived': preds_test})
output.to_csv('submissions.csv', index=False)

[0 1 0 0 1 0 1 0 1 0 0 0 1 0 1 1 0 0 1 1 0 1 1 0 1 0 1 0 0 0 0 0 1 1 0 0 1
 1 0 0 0 0 0 1 1 0 0 0 1 1 0 0 1 1 0 0 0 0 0 1 0 0 0 1 1 1 1 0 0 1 1 0 1 0
 1 0 0 1 0 1 1 0 0 0 0 0 1 1 1 1 1 0 1 0 0 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0
 1 1 1 1 0 0 1 0 1 1 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0
 0 0 1 0 0 1 0 0 1 1 0 1 1 1 1 0 0 1 0 0 1 1 0 0 0 0 0 1 1 0 1 1 0 0 1 0 1
 0 1 0 0 0 0 0 1 0 1 0 1 1 0 1 1 1 0 1 0 0 1 0 1 0 0 0 0 1 0 0 1 0 1 0 1 0
 1 0 1 1 0 1 0 0 0 1 0 0 0 0 0 0 1 1 1 1 0 0 1 0 1 0 1 1 1 0 0 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 1 0 0 0 1 1 0 1 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 1 0 0 0 0
 1 0 0 0 0 0 0 0 1 1 0 1 0 1 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0
 1 0 0 0 0 0 1 0 0 0 1 1 1 0 1 0 1 1 0 0 0 1 0 1 0 0 1 0 1 1 0 1 0 0 1 1 0
 0 1 0 0 1 1 1 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 1 0 0 1 0 1 0 0 1 0 1 0 0 0 0
 0 1 1 1 1 1 0 1 0 0 1]
